INSTALL ALBUMENTATIONS (TIVE QUE USAR ESSES ARGUMENTOS POR JÁ TER OPENCV INSTALADO NO SISTEMA)

In [2]:
!pip install -q -U albumentations --no-binary qudida,albumentations

In [ ]:
import cv2
import albumentations as A
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import csv
import random
#from google.colab.patches import cv2_imshow

import os 
import json 



OPEN CSV COM ANOTAÇÕES DE BOUNDING BOXES


In [ ]:
images    = []
trainRows = []
allRows   = []

with open('/content/drive/MyDrive/CSVs/Modelo5.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        
        allRows.append(row)
        checkTrain = row[0]

        if checkTrain == 'TRAIN':
            trainRows.append(row)

        if not images:
            if checkTrain == 'TRAIN':
                images.append(row[1])

        elif row[1] != images[-1] and checkTrain == 'TRAIN':
            images.append(row[1])

In [ ]:
def draw_labels(labels,img,show = False):
    
    labels = np.array(labels).astype(int) 
    img = np.array(img).astype(np.uint8)  
    h,w,c  = img.shape
    for bb in labels:
        if max(bb)<=1:
            y = (bb[[1,3,5,7]]*h).astype(int)
            x = (bb[[0,2,4,6]]*w).astype(int)
            cv2.rectangle(img,(x[0],y[0]),(x[1],y[2]),(0,255,0),2)
        #if bb.shape[0] == 4:
        #    bb_int = bb.astype(int)
        #    cv2.rectangle(img,(bb_int[0],bb_int[1]),(bb_int[2],bb_int[3]),(0,255,0),2)
        else:
            y = (bb[[1,3]]).astype(int)
            x = (bb[[0,2]]).astype(int)
            cv2.rectangle(img,(x[0],y[0]),(x[1],y[1]),(0,255,0),2)
        
    if show:
        plt.figure(figsize = (15,30))
        plt.imshow(img)
    
    return img

In [ ]:

def read_json(path):
    f = open(path)
    data = json.load(f)
    f.close()
    return data


def parse_labelme(data,img):
    h,w,c = img.shape
    points = []
    
    def clip(value,top):
        return min(max(value,0),top)
    
    for shapes in data["shapes"]:
        x0,y0 = np.array(shapes["points"])[0]
        x1,y1 = np.array(shapes["points"])[1]

        if x0>x1:
            x1,x0 = x0,x1 

        if y0>y1:
            y1,y0 = y0,y1 
            
        if x1 == x0 or y1 == y0:
            continue

        x1,x0,y1,y0 = clip(x1,w), clip(x0,w), clip(y1,h), clip(y0,h)
        points.append([int(x0), int(y0), int(x1), int(y1)])
    
    return np.array(points)

def labelme_to_tf(data,img):
    h,w,c = img.shape
    points = []
    
    def clip(value):
        return min(max(value,0),1)
    
    for shapes in data["shapes"]:
        x0,y0 = np.array(shapes["points"])[0]
        x1,y1 = np.array(shapes["points"])[1]
        
        if x0>x1:
            x1,x0 = x0,x1 
        
        if y0>y1:
            y1,y0 = y0,y1 
        
        x1,x0,y1,y0 = clip(x1),clip(x0),clip(y1),clip(y0)
        
        if x1 == x0 or y1 == y0:
            continue

        points.append([x0/w, y0/h,  x1/w, y0/h, x1/w, y1/h, x0/w, y1/h])
       
    
    return np.array(points)
    
    
def proportion_to_pixel(img,labels):
    h,w,c  = img.shape
    bb = []
    for proportional_bb in labels:
        #pix_bb = proportional_bb.copy()
        
        y = (proportional_bb[[1,3,5,7]]*h).astype(int)
        x = (proportional_bb[[0,2,4,6]]*w).astype(int)
        x_min, y_min, x_max, y_max = x[0],y[0],x[1],y[2]
        
        bb+=[[x_min, y_min, x_max, y_max]]
        
    return np.array(bb)
        
def pixel_to_proportion(img,labels):
    h,w,c  = img.shape
    bb = []
    for proportional_bb in labels:
        
        #pix_bb = proportional_bb.copy()
        #print("H",h,"W",w,"proportional_bb",proportional_bb)
        
        y = proportional_bb.astype(np.float32)[[0,2]]/h
        
        x = proportional_bb.astype(np.float32)[[0,2]]/w
        

        
        #print(x,y)
        
        x_min, y_min, x_max, y_max = x[0],y[0],x[1],y[1]
        
        if x_min == x_max or y_max == y_min:
            continue
        
        bb+=[[x_min, y_min, x_max, y_max]]
        
    return np.array(bb)
    




In [ ]:
folder_path = "realCampo/"

files = os.listdir(folder_path)
k=0 
for file in files:
    k+=1
    if ".json" in file:
        
        img    = cv2.imread(folder_path+file.replace(".json",".jpg"))[:,:,::-1]
        data   = read_json(folder_path+file)
        labels = labelme_to_tf(data,img)
        bboxes = proportion_to_pixel(img,labels)
        draw_labels(bboxes,img)
        
        break
        
img    = cv2.imread("realCampo/6.jpg")[:,:,::-1]
data   = read_json("realCampo/6.json")
labels = parse_labelme(data,img)
bboxes = pixel_to_proportion(img,labels)

img_drawn = draw_labels(labels,img,True)

In [ ]:
lass Change:
    def __init__(self):

        self.input_transform = A.Compose([

        A.OneOf([
            
            A.Blur(blur_limit=20, p=0.99),
            
            A.Spatter(mean=0.65, 
                          std=0.5, 
                          gauss_sigma=2, 
                          cutout_threshold=0.68, 
                          intensity=0.6, 
                          mode='rain', 
                          always_apply=True, 
                          p=0.99)

            ], p=1)

        ],bbox_params=A.BboxParams(format='pascal_voc', label_fields=[]))

    def apply(self,img,bboxes):
        
        return self.input_transform(image=img, bboxes=bboxes)


In [ ]:
change      = Change()  
transformed = change.apply(img,bboxes)
    
plt.figure(figsize = (20,10))
plt.imshow(transformed["image"])

In [ ]:
class Noise:
    def __init__(self):

        self.noise_transform = A.Compose([

        A.OneOf([


            #A.ImageCompression (quality_lower=65, quality_upper=75, always_apply=False, p=0.99), 

            #A.ISONoise(color_shift=(0.2, 0.3), intensity=(0.1, 0.2), always_apply=False, p=0.99), 

            #A.MultiplicativeNoise (multiplier=(0.7, 1.3), per_channel=True, elementwise=False, always_apply=True, p=0.99),

            A.Sharpen(alpha=(0.2, 0.25), lightness=(2, 4), always_apply=True, p=0.99),
            
            A.RGBShift(r_shift_limit=130, g_shift_limit=130, b_shift_limit=130, always_apply=True, p=0.99)

            ], p=1)

        ],bbox_params=A.BboxParams(format='pascal_voc', label_fields=[]))
        
    def apply(self,img,bboxes):
        
        return self.noise_transform(image=img, bboxes=bboxes) 

In [ ]:
noise       = Noise()
transformed = noise.apply(img,bboxes)

plt.figure(figsize = (20,10))
plt.imshow(transformed["image"])

In [ ]:
class Color:
    def __init__(self):
        
        self.color_transform = A.Compose([

        A.OneOf([


            #A.HueSaturationValue (hue_shift_limit=130, sat_shift_limit=140, val_shift_limit=30, always_apply=False, p=0.99),

            #A.RandomBrightnessContrast (brightness_limit=0.3, contrast_limit=0.05, brightness_by_max=True, always_apply=True, p=0.99)

            A.RandomGamma (gamma_limit=(170, 280), eps=None, always_apply=True, p=0.99)

            


            ], p=1)

        ],bbox_params=A.BboxParams(format='pascal_voc', label_fields=[]))


    def apply(self,img,bboxes):
        
        return self.color_transform(image=img, bboxes=bboxes)
    

In [ ]:
color = Color()

transformed = color.apply(img,bboxes)

plt.figure(figsize = (20,10))
plt.imshow(transformed["image"])

[1/i for i in range(2,6)]
4 rotações (original + 3)
4 crops (original + 3)
3 transformações (original mais 3)


In [ ]:
class Crop:
    def __init__(self,crop_proportions = None):
        self.iters = 0
        
        if crop_proportions is None:
            self.crop_proportions     = [1/i for i in range(2,6)]
            self.crop_proportions_4k  = [0.1, 0.15, 0.2, 0.24, 0.5]
            #[i for i in np.linspace( 0.1,0.8,6)]
            
    def apply(self,img,bboxes,previous_transforms):
        h,w,c  = img.shape
        self.crops = {}
        
        if h > 2000 and w > 2000:
            
            for proportion in self.crop_proportions_4k :
                
                visibility = 1 if proportion < 0.22 else 0.94 
                
                self.crops[str(round(proportion,3))] = A.Compose([
                    A.RandomCrop(width=int(w*proportion), height=int(h*proportion))
                ],bbox_params = A.BboxParams(format='pascal_voc', min_visibility=visibility, label_fields=[]))
        else:  
            for proportion in self.crop_proportions:
                
                visibility = 1 if proportion < 0.22 else 0.94 
                
                self.crops[str(round(proportion,3))] = A.Compose([
                    A.RandomCrop(width=int(w*proportion), height=int(h*proportion))
                ],bbox_params = A.BboxParams(format='pascal_voc', min_visibility=visibility, label_fields=[]))
        
        
        transformations = {}
        
        for proportion,crop in self.crops.items():

            croped = crop(image=img, bboxes=bboxes)
            
            
            tries = 0
            while len(croped["bboxes"])<3 and tries <= 200:
                croped = crop(image=img, bboxes=bboxes)
                tries+=1
                #print(len(croped["bboxes"]),"tries ",tries)
            
            if tries==201:
                print("crop tries exceed 200")
                continue
            #print(len(croped["bboxes"]))
                
            transformations[previous_transforms+"+crop:v"+str(self.iters)+"-p"+str(proportion)] = {"image":croped["image"],"bboxes":croped["bboxes"]}
                
            self.iters+=1
            
        return transformations
            

In [ ]:
class Rotations:
    def __init__(self):
        
        self.vertical = A.Compose([

            A.VerticalFlip(p=1)

        ],bbox_params = A.BboxParams(format='pascal_voc', min_visibility=0.87, label_fields=[]))

        self.horizontal = A.Compose([

            A.HorizontalFlip(p=1)

        ],bbox_params = A.BboxParams(format='pascal_voc', min_visibility=0.87, label_fields=[]))


        self.vertical_horizontal = A.Compose([

            A.VerticalFlip(p=1),
            A.HorizontalFlip(p=1)

        ],bbox_params = A.BboxParams(format='pascal_voc', min_visibility=0.87, label_fields=[]))

    def apply(self,img,bboxes,root):
        
        transformations = {}
        
        print("TOTAL LABELS:",np.array(bboxes).shape)
        
        transformations[root+"_RAW"] = {"image":img.copy(),"bboxes":np.array(bboxes).astype(int).tolist()}
        
        #vertical_fliped    = self.vertical(image=img, bboxes=bboxes)
        #transformations[root+"_vertical_fliped"] = {"image":vertical_fliped["image"],"bboxes":np.array(vertical_fliped["bboxes"]).astype(int).tolist()}
        
        #vertical_fliped    = self.horizontal(image=img, bboxes=bboxes)
        #transformations[root+"_horizontal_fliped"] = {"image":vertical_fliped["image"], "bboxes":np.array(vertical_fliped["bboxes"]).astype(int).tolist()}
        
        vertical_fliped    = self.vertical_horizontal(image=img, bboxes=bboxes)
        transformations[root+"_vertical_horizontal_fliped"] = {"image":vertical_fliped["image"],"bboxes":np.array(vertical_fliped["bboxes"]).astype(int).tolist()}
        
        return transformations


In [ ]:
img    = cv2.imread("realCampo/8.jpg")[:,:,::-1]
data   = read_json("realCampo/8.json")
labels = parse_labelme(data,img)
bboxes = pixel_to_proportion(img,labels)


In [ ]:
root   = "realCampo/8.json"
img    = cv2.imread("realCampo/8.jpg")[:,:,::-1]
data   = read_json("realCampo/8.json")
labels = parse_labelme(data,img)
bboxes = pixel_to_proportion(img,labels)

img           = draw_labels(labels,img,True)
augmentations = {}
rotations     = Rotations()
crop          = Crop()
rotated       = rotations.apply(img,labels,root)

In [ ]:
#img    = cv2.imread("realCampo/2.jpg")[:,:,::-1]
#data   = read_json("realCampo/2.json")
#labels = labelme_to_tf(data,img)
#bboxes = proportion_to_pixel(img,labels)

#img    = cv2.imread("CENIBRA/IMG_6272.JPG")[:,:,::-1]
#data   = read_json("CENIBRA/IMG_6272.json")

#img    = cv2.imread("CENIBRA/IMG_6338.JPG")[:,:,::-1]
#data   = read_json("CENIBRA/IMG_6338.json")

root   = "Eucalipto/6.json"

img    = cv2.imread("Eucalipto/6.jpg")[:,:,::-1]
data   = read_json("Eucalipto/6.json")

labels = parse_labelme(data,img)
bboxes = pixel_to_proportion(img,labels)


augmentations = {}
rotations     = Rotations()
crop          = Crop()
rotated       = rotations.apply(img,labels,root)
augmentations.update(rotated)


for key in rotated:   
    crop_transforms = crop.apply(rotated[key]["image"],rotated[key]["bboxes"],key)
    augmentations.update(crop_transforms)
    
augmentations.update(rotated) 

color = Color()

keys = list(augmentations.keys())

for key in keys:
    print(key)
    
    transformed_color  = color.apply(augmentations[key]["image"],augmentations[key]["bboxes"])
    transformed_change = change.apply(augmentations[key]["image"],augmentations[key]["bboxes"])
    transformed_noise  = noise.apply(augmentations[key]["image"],augmentations[key]["bboxes"])
    
    augmentations[key+"+color"]  = transformed_color
    augmentations[key+"+change"] = transformed_change
    augmentations[key+"+noise"]  = transformed_noise
    
    
    f, axarr = plt.subplots(1,4,figsize=(32,14))
    axarr[0].axis('off')
    axarr[1].axis('off')
    axarr[2].axis('off')
    axarr[3].axis('off')
    
    axarr[0].set_title(key)
    axarr[1].set_title(key+"_Color")
    axarr[2].set_title(key+"_Change")
    axarr[3].set_title(key+"_Noise")
    
    print(augmentations[key]["image"].shape)
    print(transformed_color["image"].shape)
    print( transformed_change["image"].shape)
    print(transformed_noise["image"].shape)
    
    draw_labels(augmentations[key]["bboxes"],  augmentations[key]["image"])
    draw_labels(transformed_color["bboxes"],   transformed_color["image"])
    draw_labels(transformed_change["bboxes"],  transformed_change["image"])
    draw_labels(transformed_noise["bboxes"],   transformed_noise["image"])
    
    #axarr[0].imshow(draw_labels(augmentations[key]["bboxes"],  augmentations[key]["image"]))
    #axarr[1].imshow(draw_labels(transformed_color["bboxes"],   transformed_color["image"]) )
    #axarr[2].imshow(draw_labels(transformed_change["bboxes"],  transformed_change["image"]))
    #axarr[3].imshow(draw_labels(transformed_noise["bboxes"],   transformed_noise["image"]) )
    
    axarr[0].imshow(draw_labels(augmentations[key]["bboxes"],  augmentations[key]["image"]))
    axarr[1].imshow(transformed_color["image"])
    axarr[2].imshow(transformed_change["image"])
    axarr[3].imshow(transformed_noise["image"])
    
    plt.show()

In [ ]:
img     = cv2.imread("Eucalipto/6.jpg")[:,:,::-1]
h, w, c = img.shape
print(h/w)
int(h*0.2) ,int(w*0.2)

resized = cv2.resize(img, (int(h*0.2),int(w*0.2)), interpolation = cv2.INTER_AREA)
h, w, c = resized.shape

print(h/w)

In [ ]:
def convert_to_tf_lite(image,bboxes,image_path):
    '''
    (x1,y1) ---------------  (x2,y2)
      |                         |
      |                         |
      |                         |
      |                         |
    (x3,y3) ---------------- (x4,y4)

    '''

    bboxes  = bboxes
    h, w, c = image.shape
    
    tf_bounding_boxes = []
    for bb in bboxes:
        x_min, y_min, x_max, y_max = bb
        
        #            "x1",        "x2",          "x3",        "x4",
        #bb = [float(x_min), float(x_max), float(x_min), float(x_max), 
        #       "y1",              "y2",        "y3",          "y4"
        #      float(y_min), float(y_min), float(y_max), float(y_max)]
        
        
         #            "x1",        "y1",                "x2",        "y1",
        bb = [float(x_min), float(y_min),   float(x_max), float(y_min), 
        #           "x2",       "y2",          "y3",          "y4"
              float(x_max), float(y_max),   float(x_min), float(y_max)]

        bb = np.array(bb).astype(np.float16)
        
        bb[[1,3,5,7]] = (bb[[1,3,5,7]]/h).astype(np.float16)
        bb[[0,2,4,6]] = (bb[[0,2,4,6]]/w).astype(np.float16)
        
        
        tf_bounding_boxes.append(bb)
        
        #["split", "path", "log","x1","x2","x3","x4","y1","y2","y3","y4"]
    return np.array(tf_bounding_boxes)


In [ ]:
def show_detections(augmentations,transformed_color,key):
    f, axarr = plt.subplots(1,4,figsize=(32,14))
    axarr[0].axis('off')
    axarr[1].axis('off')
    axarr[2].axis('off')
    axarr[3].axis('off')

    axarr[0].set_title(key)
    axarr[1].set_title(key+"_Color")
    axarr[2].set_title(key+"_Change")
    axarr[3].set_title(key+"_Noise")

    #draw_labels(augmentations[key]["bboxes"],  augmentations[key]["image"])
    #draw_labels(transformed_color["bboxes"],   transformed_color["image"])
    #draw_labels(transformed_change["bboxes"],  transformed_change["image"])
    #draw_labels(transformed_noise["bboxes"],   transformed_noise["image"])
    #axarr[0].imshow(draw_labels(augmentations[key]["bboxes"],  augmentations[key]["image"]))
    #axarr[1].imshow(draw_labels(transformed_color["bboxes"],   transformed_color["image"]) )
    #axarr[2].imshow(draw_labels(transformed_change["bboxes"],  transformed_change["image"]))
    #axarr[3].imshow(draw_labels(transformed_noise["bboxes"],   transformed_noise["image"]) )

    axarr[0].imshow(draw_labels(augmentations[key]["bboxes"],  augmentations[key]["image"]))
    axarr[1].imshow(draw_labels(transformed_color["bboxes"],   transformed_color["image"]))
    axarr[2].imshow(draw_labels(transformed_change["bboxes"],  transformed_change["image"]))
    axarr[3].imshow(draw_labels(transformed_noise["bboxes"],   transformed_noise["image"]))

    plt.show()

In [ ]:
from tqdm import tqdm
import pandas as pd

def AugumentationPipeline(label_paths,show = False):
    
    image_path = label_paths.replace(".json",".jpg")
    root       = label_paths.replace(".json","").replace("/","_")
    
    img    = cv2.imread(image_path)
    data   = read_json(label_paths)
    labels = parse_labelme(data,img)
    bboxes = pixel_to_proportion(img,labels)

    augmentations = {}
    rotations     = Rotations()
    crop          = Crop()
    color         = Color()
    change        = Change()
    noise         = Noise()
    
    rotated       = rotations.apply(img,labels,root)
    
    print("applying rotations")
    augmentations.update(rotated)
    
    print("applying crop")
    
    for key in tqdm(rotated): 
        crop_transforms = crop.apply(rotated[key]["image"],rotated[key]["bboxes"],key)
        augmentations.update(crop_transforms)
    
    keys = list(augmentations.keys())
    
    print("applying change color and noise augumentations")
    
    for key in tqdm(keys):
        
        transformed_color  = color.apply(augmentations[key]["image"] ,augmentations[key]["bboxes"])
        transformed_change = change.apply(augmentations[key]["image"],augmentations[key]["bboxes"])
        transformed_noise  = noise.apply(augmentations[key]["image"] ,augmentations[key]["bboxes"])

        augmentations[key+"+color"]  = transformed_color
        augmentations[key+"+change"] = transformed_change
        augmentations[key+"+noise"]  = transformed_noise
    
        if show:
            show_detections(augmentations,transformed_color,key)
            
    save_path = "augumentations/"+label_paths.replace(".json","").split("/")[0]
    
    if not os.path.exists(save_path):
        os.mkdir(save_path)
        
    df_dict = {
        "split": [],
        "path":  [],
        "log":   [],
        "x1":    [],
        "x2":    [],
        "x3":    [],
        "x4":    [],
        "y1":    [],
        "y2":    [],
        "y3":    [],
        "y4":    []
    } 
    
    print("saving images and labels")
    
    prob = np.random.rand()
    
    if prob < 0.7:
        split = "TRAIN"   
    elif prob < 0.9:
        split = "TEST" 
    else: 
        split = "VALIDATION"
    
    for key in tqdm(list(augmentations.keys())):
        
        if len(augmentations[key]["bboxes"])>300: 
            print("too many labels in image",len(augmentations[key]["bboxes"]))
            continue
        
        image_path = save_path + "/" + key+ ".jpg"
        cv2.imwrite(image_path, augmentations[key]["image"])
        
        h,w,c      = augmentations[key]["image"].shape
        tf_lite_bb = convert_to_tf_lite(augmentations[key]["image"],augmentations[key]["bboxes"],image_path)
        
        for bb in tf_lite_bb:
            #print(bb)
            df_dict["split"].append(split)
            df_dict["path"].append(image_path)
            df_dict["log"].append("log") 
            df_dict["x1"].append(bb[0])  
            df_dict["x2"].append(bb[1])   
            df_dict["x3"].append(bb[2])   
            df_dict["x4"].append(bb[3])   
            df_dict["y1"].append(bb[4])   
            df_dict["y2"].append(bb[5])   
            df_dict["y3"].append(bb[6])   
            df_dict["y4"].append(bb[7]) 
        
    return pd.DataFrame(df_dict)   
        

In [ ]:
import os 
from random import shuffle

def generate_all_label_paths():
    
    folders =   ['Eucalipto',
                 'oldPinus',
                 'CENIBRA',
                 'realCampo',
                 'Pinus']

    label_paths = []

    for folder in folders:
        files = os.listdir(folder)
        for file in files:
            if ".json" in file:
                label_paths.append(folder+"/"+file)

    for label in label_paths:
        image_path = label.replace(".json",".jpg")
        
    return label_paths

labels = generate_all_label_paths()

shuffle(labels)

len(labels)

In [ ]:
df = None

for i,label in enumerate(labels):
    try:
        print("Augumenting image:",i,"out of", len(labels),"completed",round(i/len(labels)*100,2),"%", "path",label)
        if df is None:
            df = AugumentationPipeline(label,show = False)

        else:
            df2 = AugumentationPipeline(label,show = False)
            df = pd.concat([df,df2])

        df.to_csv("whole.csv",header=False, index=False)
    except Exception as e:
        print(e)



In [ ]:
df

In [ ]:
df.to_csv("whole.csv",header=False, index=False)

In [ ]:
names = AugumentationPipeline(label_paths[0],show = False)

In [ ]:
A.VerticalFlip(p=0.5),
A.HorizontalFlip(p=0.5),

transform = A.Compose([

    #A.RandomCrop(width=int(w/i), height=int(h/i))

],bbox_params = A.BboxParams(format='pascal_voc', min_visibility=0.77, label_fields=[]))

In [ ]:
4*4*4*80

In [ ]:
labels.shape,np.array(transformed_bboxes).shape

In [ ]:
#img                  = draw_labels(labels,img)
transformed_draw_img = draw_labels(np.array(transformed_bboxes),transformed_img)

f, axarr = plt.subplots(1,2,figsize=(32,14))
axarr[0].axis('off')
axarr[1].axis('off')
axarr[0].set_title("Raw")
axarr[1].set_title("Transformed")
axarr[0].imshow(img[:,:,::-1])
axarr[1].imshow(transformed_draw_img[:,:,::-1])

In [ ]:

pastaDrive = '/content/drive/MyDrive/images_16.02.22/augmentation'



for image in images:
    split = image.split("/")
    nomePasta = split[5]
    nomeFoto = split[6]

    bboxes = []


    for row in trainRows:
    if image == row[1]:
        temp = []
        temp.append(float(row[3]))
        temp.append(float(row[4]))
        temp.append(float(row[7]))
        temp.append(float(row[8]))
        bboxes.append(temp)

    img = cv2.imread(image)


    transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.OneOf([
        A.Blur(p=0.3),
        A.MedianBlur(p=0.3),
        A.MotionBlur(p=0.3),
    ], p=0.8),
    A.OneOf([
        A.RandomBrightnessContrast(p=1.0),
    ], p=0.8),

    # A.ISONoise(p=1.0),
    # A.RandomRain(p=1.0, drop_length=27, blur_value=5, brightness_coefficient=0.7)
    # A.RandomSunFlare(p=1.0)
    # A.Perspective(p=1.0)
    ],bbox_params=A.BboxParams(format='albumentations', label_fields=[]))


    images_list = []
    bboxes_list = []

    for i in range(3):
        transformed = transform(image=img, bboxes=bboxes)
        transformed_img = transformed["image"]
        transformed_bboxes = transformed["bboxes"]

        images_list.append(transformed_img)
        bboxes_list.append(transformed_bboxes)


    photoCount = 1
    j = 0
    for imageT in images_list:
        nomeFotoT = pastaDrive + '/' + nomePasta + '/' + nomeFoto[:-4] + '_T' + str(photoCount) + '.jpg'
        cv2.imwrite(nomeFotoT, imageT)
        height = imageT.shape[0]
        width = imageT.shape[1]
        for bboxes in bboxes_list[j]:  
            x_min = bboxes[0]
            y_min = bboxes[1]
            x_max = bboxes[2]
            y_max = bboxes[3]

            ymin = int(max(1,(y_min * height)))
            xmin = int(max(1,(x_min * width)))
            ymax = int(min(height,(y_max * height)))
            xmax = int(min(width,(x_max * width)))

            data = ["TRAIN", nomeFotoT, 'log', x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max]

            allRows.append(data)

            cv2.rectangle(imageT, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)


    photoCount += 1
    j += 1
    # cv2_imshow(imageT)


SHUFFLE NO DATASET

In [ ]:
newUniqueRows = []
newRowsCount = []
newRows = []


for x in allRows:

    newRowsCount.append(x)

    linha = x[1]

    if not newUniqueRows:
        newUniqueRows.append(linha)

    elif newUniqueRows[-1] != linha:
        newUniqueRows.append(linha)


#shuffle
random.shuffle(newUniqueRows)


for index, i in enumerate(newUniqueRows):
    for j in newRowsCount:
        if i == j[1]:
            data = [j[0], j[1], j[2], j[3], j[4], j[5], j[6], j[7], j[8], j[9], j[10]]
            newRows.append(data)

SALVA CSV COM TODAS AS BOUNDING BOXES


In [ ]:
with open('/content/drive/MyDrive/CSVs/Modelo5_T.csv', 'w', newline='') as f:

    for data in newRows:

        writer = csv.writer(f)

        writer.writerow(data)